In [1]:
from sklearn import datasets
iris_dataset = datasets.load_iris()
x = iris_dataset.data
print(x)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.

In [2]:
y = iris_dataset.target

In [3]:
from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
x = scaler_x.fit_transform(x)

In [4]:
from sklearn.preprocessing import OneHotEncoder
oneHot = OneHotEncoder()

In [5]:
y = y.reshape(-1, 1)

In [6]:
y = oneHot.fit_transform(y).toarray()

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [8]:
import tensorflow as tf
import numpy as np

In [9]:
input_neurons = x.shape[1]
input_neurons

4

In [10]:
ouput_hidden_neurons = int(np.ceil((x.shape[1] + y.shape[1])/2))

In [11]:
output_neurons = y.shape[1]

In [12]:
weights = {
'hidden': tf.Variable(tf.random.normal([input_neurons, ouput_hidden_neurons]), name='hidden_weights'),
'output': tf.Variable(tf.random.normal([ouput_hidden_neurons, output_neurons]), name='output_weights')
}

In [13]:
bias = {
    'hidden': tf.Variable(tf.random.normal([ouput_hidden_neurons]), name='hidden_bias'),
    'output': tf.Variable(tf.random.normal([output_neurons]), name='output_bias')
}


In [14]:
x_train = tf.convert_to_tensor(x_train, dtype = tf.float32, name = 'x_train')
y_train = tf.convert_to_tensor(y_train, dtype = tf.float32, name = 'y_train')
x_test = tf.convert_to_tensor(x_test, dtype = tf.float32, name = 'x_test')
y_test = tf.convert_to_tensor(y_test, dtype = tf.float32, name = 'y_test')

In [15]:
def mlp(x, weights, bias):
    hidden_layer = tf.add(tf.matmul(x, weights['hidden']), bias['hidden'])
    hidden_layer_activation = tf.nn.relu(hidden_layer)
    output_layer = tf.add(tf.matmul(hidden_layer_activation, weights['output']), bias['output'])
    return output_layer

In [16]:
with tf.GradientTape() as tape:
    model = mlp(x_train, weights, bias)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_train, logits = model))

In [20]:
var_list = [weights['hidden'], weights['output'], bias['hidden'], bias['output']]

In [21]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001)
gradients = tape.gradient(loss, var_list)
optimizer.apply_gradients(zip(gradients, var_list))

<tf.Variable 'UnreadVariable' shape=() dtype=int64, numpy=1>

In [24]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)

In [28]:
for epoca in range(5000):
    for x_batch, y_batch in dataset:
        standard_error = 0.0
        with tf.GradientTape() as tape:
            model = mlp(x_batch, weights, bias)
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_batch, logits = model))
        gradients = tape.gradient(loss, var_list)
        optimizer.apply_gradients(zip(gradients, var_list))
        standard_error += loss.numpy()
    if epoca % 500 == 0:
        print('Época: ' + str((epoca)) + ' erro: ' + str(standard_error))

Época: 0 erro: 0.05829063057899475
Época: 500 erro: 0.05323277413845062
Época: 1000 erro: 0.04907745495438576
Época: 1500 erro: 0.045206744223833084
Época: 2000 erro: 0.038587916642427444
Época: 2500 erro: 0.034782446920871735
Época: 3000 erro: 0.03204023092985153
Época: 3500 erro: 0.02969270385801792
Época: 4000 erro: 0.027629828080534935
Época: 4500 erro: 0.025790544226765633


In [ ]:
predict_test = mlp(x_test, weights, bias)
predict_test = tf.nn.softmax(predict_test)
predict_test = tf.argmax(predict_test, axis = 1).numpy()
y_teste2 = tf.argmax(y_test, axis = 1).numpy()
from sklearn.metrics import accuracy_score
print(accuracy_score(y_teste2, predict_test))